# Data Exploration and Visualization of Cosmic Strings in CMB
This notebook explores the Cosmic Microwave Background (CMB) data to identify potential cosmic string candidates.

Initial exploration of the CMB data using Healpy and Matplotlib.

In [ ]:
# Install required packages
!pip install healpy matplotlib astropy scipy scikit-image

First exploring the COM_CMB_IQU-smica_2048_R3.00_full.fits file to understand its structure and contents.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.constants.codata2014 import alpha
from scipy.constants import h, c, k
import healpy as hp

# Planck function (spectral radiance)
def planck(f, T):
    x = h * f / (k * T)
    return (2 * h * f**3 / c**2) / np.expm1(x)


# Frequency range (Hz)
frequencies = np.linspace(1e10, 1e12, 1000)  # 10 GHz to 1000 GHz
temperature = 2.725  # CMB temperature in Kelvin

# Compute spectral radiance
radiance = planck(frequencies, temperature)

# Plot spectrum
plt.figure(figsize=(10, 6))
plt.plot(frequencies / 1e9, radiance, color='darkblue', label=f'T = {temperature} K')
plt.title('CMB Blackbody Spectrum at T = 2.725 K')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Spectral Radiance B(ν) [W·sr⁻¹·m⁻²·Hz⁻¹]')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:

from scipy import ndimage

# Load a Planck CMB temperature map
cmb_map = hp.read_map('data/COM_CMB_IQU-smica_2048_R3.00_full.fits', field=0)

# Not ideal for CMB structure detection
# cmb_map = hp.read_map('data/HFI_SkyMap_100_2048_R3.01_full.fits', field=0)


# Convert to 2D image
nside = hp.get_nside(cmb_map)
npix = hp.nside2npix(nside)
# Convert the CMB map to a 2D image for visualization
img = hp.mollview(cmb_map, return_projected_map=True, nest=False, title='CMB Temperature Map', cmap='inferno', xsize=2000, hold=True)


In [ ]:
# Apply edge detection (e.g., Sobel)edges = ndimage.sobel(img)
# Plot the edges detected in the CMB map
edges = ndimage.sobel(img)

from pathlib import Path

folder_path = Path('./edge')

if not folder_path.exists():
    folder_path.mkdir(parents=True, exist_ok=True)
    print(f"Folder '{folder_path}' created successfully!")
else:
    print(f"Folder '{folder_path}' already exists.")

# Save to .fits
from astropy.io import fits
# Save as 2D FITS image
hdu = fits.PrimaryHDU(edges.astype(np.float32))
hdu.writeto("edge/edge_detected_image.fits", overwrite=True)


plt.figure(figsize=(30, 15))
plt.imshow(edges, cmap='inferno')
plt.title('Possible Cosmic String Candidates (Edges in CMB)')
plt.colorbar()
plt.show()


In [ ]:
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt

# --- Step 1: Load or define your image (already done)
# img = ...

# --- Step 2: Force-clean the CMB image ---
# Replace NaNs, -inf, +inf with zeros (or median)
img_cleaned = np.nan_to_num(img, nan=0.0, posinf=0.0, neginf=0.0)

# --- Step 3: Edge detection ---
edges = ndimage.sobel(img_cleaned)

# --- Step 4: Print CLEANED ranges ---
print("Cleaned CMB image range:", np.min(img_cleaned), "to", np.max(img_cleaned))
print("Edge image range:", np.min(edges), "to", np.max(edges))

# --- Step 5: Normalize for plotting ---
def normalize(arr):
    arr = np.nan_to_num(arr, nan=0.0, posinf=0.0, neginf=0.0)
    return (arr - np.min(arr)) / (np.ptp(arr) + 1e-8)

img_norm = normalize(img_cleaned)
edges_norm = normalize(edges)

# Optional: Threshold edges for clarity
edges_thresh = np.where(edges_norm > 0.3, 1.0, 0.0)

# --- Step 6: Plot overlay ---
plt.figure(figsize=(14, 6))
plt.imshow(img_norm, cmap='coolwarm',alpha=0.9, origin='lower', aspect='auto')
plt.imshow(edges_thresh, cmap='YlOrBr', alpha=0.9, origin='lower', aspect='auto')
plt.colorbar(label='Intensity')
plt.title("CMB with Edge Overlay (cleaned)")
plt.axis('off')
plt.tight_layout()
plt.savefig("cmb_edge_overlay_cleaned.png", dpi=300)
plt.show()


In [ ]:
# Nonlinear exaggeration
edges_exaggerated = edges**0.9  # or try 2.0


plt.figure(figsize=(30, 15))
plt.imshow(edges_exaggerated, cmap='turbo')
plt.title('Possible Cosmic String Candidates (Edges in CMB)')
plt.colorbar()
plt.show()

In [ ]:
sharp_edges = edges + 0.7 * edges  # amplify edge intensity


plt.figure(figsize=(30, 15))
plt.imshow(sharp_edges, cmap='inferno')
plt.title('Possible Cosmic String Candidates (Edges in CMB)')
plt.colorbar()
plt.show()


In [ ]:
# Alter this code block
# Apply edge detection (e.g., Sobel)edges = ndimage.sobel(img)
# Plot the edges detected in the CMB map
# Apply Sobel in x and y directions
dx = ndimage.sobel(img, axis=0) ** 0.5
dy = ndimage.sobel(img, axis=1) ** 0.5

# Gradient magnitude
edges = np.hypot(dx, dy)  # Same as sqrt(dx**2 + dy**2)

plt.figure(figsize=(30, 15))
plt.imshow(edges, cmap='inferno')
plt.title('Possible Cosmic String Candidates (Edges in CMB)')
plt.colorbar()
plt.show()


In [ ]:
frequencies = np.linspace(10e9, 1000e9, 1000)  # 10 GHz to 1000 GHz

T_mean = 2.725
delta_T = 100e-6  # 100 µK typical fluctuation

plt.figure(figsize=(10, 6))
plt.plot(frequencies / 1e9, planck(frequencies, T_mean), label="T = 2.725 K", color='blue')
plt.plot(frequencies / 1e9, planck(frequencies, T_mean + delta_T), '--', label="+100 µK", color='green', alpha=0.8)
plt.plot(frequencies / 1e9, planck(frequencies, T_mean - delta_T), '--', label="-100 µK", color='red', alpha=0.8)
plt.xlabel("Frequency (GHz)")
plt.ylabel("Spectral Radiance (W·sr⁻¹·m⁻²·Hz⁻¹)")
plt.title("CMB Spectrum with Fluctuation Bounds (±100 µK)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Show the CMB spectrum with +100 µK fluctuations
plt.figure(figsize=(10, 6))
plt.plot(frequencies / 1e9, planck(frequencies, T_mean), label="T = 2.725 K", color='blue')
plt.plot(frequencies / 1e9, planck(frequencies, T_mean + delta_T), '--', label="+100 µK", color='green', alpha=0.8)
plt.xlabel("Frequency (GHz)")
plt.ylabel("Spectral Radiance (W·sr⁻¹·m⁻²·Hz⁻¹)")
plt.title("CMB Spectrum with Fluctuation Bounds (±100 µK)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Show the CMB spectrum with -100 µK fluctuations
plt.figure(figsize=(10, 6))
plt.plot(frequencies / 1e9, planck(frequencies, T_mean), label="T = 2.725 K", color='blue')
plt.plot(frequencies / 1e9, planck(frequencies, T_mean - delta_T), '--', label="-100 µK", color='red', alpha=0.8)
plt.xlabel("Frequency (GHz)")
plt.ylabel("Spectral Radiance (W·sr⁻¹·m⁻²·Hz⁻¹)")
plt.title("CMB Spectrum with Fluctuation Bounds (±100 µK)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
cl = hp.anafast(cmb_map)
plt.plot(cl[:200])
plt.title("Low-ℓ Multipoles – Texture Signature Region")
plt.xlabel("Multipole ℓ [ℓ: inverse angular scale]")
plt.ylabel("C_ℓ [Power: variance of temperature fluctuations]")
plt.grid()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
from matplotlib.ticker import FuncFormatter

# --- Compute power spectrum from CMB map ---
cl = hp.anafast(cmb_map)         # Power spectrum
ell = np.arange(len(cl))         # Multipole indices

# --- Begin Plot ---
plt.figure(figsize=(12, 6))

# Plot measured CMB power spectrum (excluding monopole)
plt.loglog(ell[1:1500], cl[1:1500], label='CMB Power Spectrum', color='blue')

# Plot theoretical scale-invariant reference curve
ref_value = np.mean(cl[5:15])
theory_curve = ref_value * (ell[5] * (ell[5] + 1)) / (ell[1:1500] * (ell[1:1500] + 1))
plt.loglog(ell[1:1500], theory_curve, 'r--', label='Scale-invariant (∝ 1/ℓ(ℓ+1))')

# --- Highlight Regions ---
plt.axvspan(2, 30, color='gray', alpha=0.15, label='Sachs-Wolfe Region')
plt.axvspan(50, 200, color='orange', alpha=0.08, label='Transition Region')
plt.axvspan(200, 1500, color='green', alpha=0.08, label='Acoustic Peaks Region')

# --- Annotations (repositioned to avoid overlap) ---
plt.text(7, 2e-10, 'Sachs-Wolfe\nSuper-horizon modes', fontsize=10, color='black')
plt.text(60, 5e-20, 'Transition:\nEarly Oscillations &\n Projection Effects', fontsize=10, color='black')
plt.text(300, 5e-12, 'Acoustic Peaks:\nPhoton-Baryon Oscillations', fontsize=10, color='black')

# 1st Peak marker
plt.axvline(220, color='black', linestyle='--', alpha=0.5)
plt.text(235, 1e-10, '1st Peak\n(~ℓ=220)', fontsize=9, color='black')

# --- Labels, Legend, Grid ---
plt.title("CMB Angular Power Spectrum (Log Scale)")
plt.xlabel("Multipole ℓ [ℓ: inverse angular scale]")
plt.ylabel("C_ℓ [Power: variance of temperature fluctuations]")
plt.legend()
plt.grid(True, which="both", ls="--", alpha=0.2)

# --- Format axes using human-readable numbers ---
def human_format(x, pos):
    if x >= 1:
        return f"{int(x)}"
    else:
        return f"{x:.1g}"

ax = plt.gca()
ax.xaxis.set_major_formatter(FuncFormatter(human_format))
ax.yaxis.set_major_formatter(FuncFormatter(human_format))

plt.tight_layout()
plt.show()


In [ ]:
# Plot the power spectrum (ℓ = 0 to 199)
ell = np.arange(len(cl))  # Define ell as the array of multipole indices
plt.figure(figsize=(9, 5))
plt.plot(ell[:250], cl[:250], label='CMB Power')

# Overlay topological defect regions
plt.axvspan(2, 10, color='blue', alpha=0.3, label='Textures (ℓ ≈ 2–10)')
plt.axvspan(2, 5, color='red', alpha=0.3, label='Domain Walls (ℓ ≈ 2–5)')
plt.axvspan(50, 200, color='green', alpha=0.2, label='Cosmic Strings (ℓ ≈ 50–200)')
plt.axvspan(2, 3, color='orange', alpha=0.4, label='Monopoles (ℓ ≈ 2–3)')

# Labels and grid
plt.title("CMB Multipole Spectrum with Topological Defect Regions")
plt.xlabel("Multipole ℓ [ℓ: inverse angular scale]")
plt.ylabel("C_ℓ [Power: variance of temperature fluctuations]")
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Plot the power spectrum (ℓ = 0 to 199)
ell = np.arange(len(cl))  # Define ell as the array of multipole indices
plt.figure(figsize=(9, 5))
plt.plot(ell[:50], cl[:50], label='CMB Power')

# Overlay topological defect regions
plt.axvspan(2, 10, color='blue', alpha=0.3, label='Textures (ℓ ≈ 2–10)')
plt.axvspan(2, 5, color='red', alpha=0.3, label='Domain Walls (ℓ ≈ 2–5)')
plt.axvspan(2, 3, color='orange', alpha=0.4, label='Monopoles (ℓ ≈ 2–3)')

# Labels and grid
plt.title("CMB Multipole Spectrum with Topological Defect Regions")
plt.xlabel("Multipole ℓ [ℓ: inverse angular scale]")
plt.ylabel("C_ℓ [Power: variance of temperature fluctuations]")
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Use same `cl` from previous block
ell = np.arange(1, 200)  # Skip ℓ = 0 to avoid divide-by-zero
theta_deg = 180 / ell    # Angular scale in degrees

# Trim cl accordingly
cl_trim = cl[1:200]

# Plot C_ell vs angular scale θ
plt.plot(theta_deg, cl_trim)
plt.title("CMB Power Spectrum vs Angular Scale")
plt.xlabel("Angular Scale θ [degrees]")
plt.ylabel("C_ℓ [Power: variance of temperature fluctuations]")
plt.grid()
plt.gca().invert_xaxis()  # Larger angular scales on the left
plt.show()

In [ ]:
# ℓ = 1 to 199 (skip ℓ = 0 to avoid division by zero)
theta_deg = 180 / ell
cl_trim = cl[1:200]

# Plot power vs angular scale
plt.figure(figsize=(9, 5))
plt.plot(theta_deg, cl_trim, label='CMB Power')

# Overlay topological defect regions in degrees
plt.axvspan(18, 90, color='blue', alpha=0.3, label='Textures (θ ≈ 18° to 90°)')
plt.axvspan(36, 90, color='red', alpha=0.3, label='Domain Walls (θ ≈ 36° to 90°)')
plt.axvspan(1, 4, color='green', alpha=0.2, label='Cosmic Strings (θ ≈ 1° to 4°)')
plt.axvspan(60, 90, color='orange', alpha=0.4, label='Monopoles (θ ≈ 60° to 90°)')

# Labels and formatting
plt.title("CMB Power vs Angular Scale with Topological Defect Regions")
plt.xlabel("Angular Scale θ [degrees]")
plt.ylabel("C_ℓ [Power: variance of temperature fluctuations]")
plt.grid()
plt.gca().invert_xaxis()  # Large scales (low ℓ) on left
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
from skimage import draw  # Changed import to use skimage.draw instead of matplotlib.pyplot.draw

def simulate_cosmic_strings(map_size, num_strings):
    map_data = np.zeros((map_size, map_size))
    for _ in range(num_strings):
        x = np.random.randint(0, map_size)
        y = np.random.randint(0, map_size)
        dx = np.random.randint(-map_size//4, map_size//4)
        dy = np.random.randint(-map_size//4, map_size//4)
        rr, cc = draw.line(x, y, x+dx, y+dy)  # Now using skimage.draw.line
        map_data[rr % map_size, cc % map_size] += np.random.choice([-1, 1]) * 100e-6  # ~100 µK jump
    return map_data

# Simulate and plot
map_size = 1000  # Reduced size for better visualization
num_strings = 10
cosmic_map = simulate_cosmic_strings(map_size, num_strings)

plt.figure(figsize=(10, 10))
plt.imshow(cosmic_map, cmap='RdBu_r')
plt.colorbar(label='Temperature fluctuation (K)')
plt.title('Simulated Cosmic Strings')
plt.show()
